# Beispiel Qualitätstest ohne unittest-Framework
In diesem Notebook wird ein Test gezeigt, bei dem nicht auf das unittest-Framework zurückgegriffen wird. Der Test ist sehr ausführlich und entsprechend kommentiert, da er instruktiven Charakter haben soll.

In [ ]:
import ifcopenshell
from ifcopenshell.api import run
import ifcopenshell.util.element
import ifcopenshell.util.selector
import re
from collections import Counter

In [2]:
# Laden der IFC-Datei.
model = ifcopenshell.open("../project_data/script_output_4x3.ifc")
model = ifcopenshell.open("../project_data/script_output_4x3_with_errors.ifc")

In [3]:
def Modellaufbau_Borehole_GeotechnicalStratum(model):
    """
    Beschreibung:
        Ein IfcBorehole dient als Container für verschiedene IfcGeotechnicalStratum. IfcBorehole beschreibt die Bohrung als ganzes, wohin IfcGeotechnicalStratum zur Repräsentation von Ansprachebereichen verwendet wird.
    Anweisung:
        Der Modellaufbau ist durch den Fachplaner für Geotechnik sicherzustellen.
    """

    # Filtern der Elemente
    elems = model.by_type("IfcGeotechnicalStratum")
    elems = [i for i in elems if i.ObjectType == "ANSPRACHEBEREICH"]
    
    # Alternativ unter Nutzung der Query Syntax
    elems = ifcopenshell.util.selector.filter_elements(model, "IfcGeotechnicalStratum, ObjectType=ANSPRACHEBEREICH")

    # Für jedes Element: Prüfen der Anforderung
    passed, failed = [], []
    for elem in elems:
        if not elem.Decomposes:
            failed.append(elem)
            continue
        if any((i.RelatingObject.is_a("IfcBorehole") and i.is_a("IfcRelAggregates")) for i in elem.Decomposes): # Hinweis: Hier muss minndestens eine der IfcRelAggregates auf ein IfcBorehole verweisen. Wenn es genau eins sein soll, dann len(elem.Decomposes) == 1 und elem.Decomposes[0] testen
            passed.append(elem)
        else:
            failed.append(elem)
    
    print(f"{len(passed)} Elemente haben bestanden, {len(failed)} sind durchgefallen.")
    if failed:
        print(f"Durchgefallen sind:\n")
        for i in failed:
            print(f"\t{failed}")

    return passed, failed

Modellaufbau_Borehole_GeotechnicalStratum(model)    


15 Elemente haben bestanden, 1 sind durchgefallen.
Durchgefallen sind:

	[#58686=IfcGeotechnicalStratum('27mPO3VuD2_RgFB2AYPh7k',$,'wrong_borehole_name_xy',$,'ANSPRACHEBEREICH',$,$,$,.USERDEFINED.)]


([#176=IfcGeotechnicalStratum('31mCmTzGDF6OQLeIXkJk_b',$,'bh_003_0',$,'ANSPRACHEBEREICH',#412,#189,$,.USERDEFINED.),
  #318=IfcGeotechnicalStratum('18lAReeSL14O5n26h3H27R',$,'bh_005_1',$,'ANSPRACHEBEREICH',#457,#331,$,.USERDEFINED.),
  #112=IfcGeotechnicalStratum('1ea67PoivCdh5u_9dbkNu_',$,'bh_002_0',$,'ANSPRACHEBEREICH',#437,#125,$,.USERDEFINED.),
  #332=IfcGeotechnicalStratum('007gFEIM5B$OGJ_gBzBjGk',$,'bh_005_2',$,'ANSPRACHEBEREICH',#462,#345,$,.USERDEFINED.),
  #254=IfcGeotechnicalStratum('1g8_OC6$j9u9oD41FO6Qx6',$,'bh_004_1',$,'ANSPRACHEBEREICH',#382,#267,$,.USERDEFINED.),
  #48=IfcGeotechnicalStratum('1di9z_UxT8sudYhkRZVe35',$,'bh_001_0',$,'ANSPRACHEBEREICH',#392,#61,$,.USERDEFINED.),
  #190=IfcGeotechnicalStratum('2$oDgFjTn2lvAIeZDQVnlc',$,'bh_003_1',$,'ANSPRACHEBEREICH',#422,#203,$,.USERDEFINED.),
  #126=IfcGeotechnicalStratum('05ywY0M952VhKv5u$2mqMG',$,'bh_002_1',$,'ANSPRACHEBEREICH',#432,#139,$,.USERDEFINED.),
  #268=IfcGeotechnicalStratum('3Mfha8MSHBEv_R_haw1g84',$,'bh_004_2

In [4]:
def WerteBereich_Kohaesion(model, min_val=0, max_val=1_000):
    """
    Prüft, ob die Eigenschaft CohesionBehaviour im Pset_SolidStratumCapacity in einem Wertebereich liegt
    """
    # Filtern der Elemente
    # Hinweis: IfcSimpleProperties können auf mehrere Arten beschrieben werden, die am häufigst verwendete ist IfcPropertySingleValue
    elems = model.by_type("IfcSimpleProperty")
    #elems = model.by_type("IfcPropertySingleValue") + model.by_type("IfcPropertyBoundedValue") + model.by_type("IfcPropertyEnumeratedValue") + model.by_type("IfcPropertyListValue") + model.by_type("IfcPropertyReferenceValue") + model.by_type("IfcPropertyTableValue")

    elems = [i for i in elems if i.Name=="CohesionBehaviour"]
    elems = [i for i in elems if any(j.Name=="Pset_SolidStratumCapacity" for j in i.PartOfPset)]

    # Für jedes Element: Prüfen der Anforderung
    passed, failed = [], []
    for elem in elems:
        value = elem.NominalValue.wrappedValue
        if value>=min_val and value<=max_val: # Hinweis: Im IDS Beispiel werden die beiden Tests gesondert gelistet.
            passed.append(elem)
        else:
            failed.append(elem)

    #    print(elem.NominalValue, elem.Unit)
    #    print(elem.HasConstraints)
    #print(dir(elem.NominalValue))
    #print(elem.NominalValue.wrappedValue)


WerteBereich_Kohaesion(model)


In [5]:
# Each IfcBorehole shall have the Pset_BoreholeCommon
elems = model.by_type("IfcBorehole")
passed, failed = [], []
for elem in elems:
    psets = ifcopenshell.util.element.get_psets(elem)
    if "Pset_BoreholeCommon" in ifcopenshell.util.element.get_psets(elem).keys():
        passed.append(elem)
    else:
        failed.append(elem)

In [6]:
# Get the containers for each IfcBorehole
elems = model.by_type("IfcBorehole")
passed, failed = [], []
for elem in elems:
    container = ifcopenshell.util.element.get_container(elem)
    if container:
        print(f"The Borehole {elem.Name} is located in {container.Name}, {container}")
    else:
        print(f"No container for {elem.Name}")

The Borehole bh_001 is located in Baustelle_Fachsektionstage, #13=IfcSite('0BkGq0KRz1vPCdK7VfEEms',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_002 is located in Baustelle_Fachsektionstage, #13=IfcSite('0BkGq0KRz1vPCdK7VfEEms',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_003 is located in Baustelle_Fachsektionstage, #13=IfcSite('0BkGq0KRz1vPCdK7VfEEms',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_004 is located in Baustelle_Fachsektionstage, #13=IfcSite('0BkGq0KRz1vPCdK7VfEEms',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
The Borehole bh_005 is located in Baustelle_Fachsektionstage, #13=IfcSite('0BkGq0KRz1vPCdK7VfEEms',$,'Baustelle_Fachsektionstage',$,$,$,$,$,$,$,$,$,$,$)
No container for wrong_borehole_name
No container for wrong_borehole_name


In [7]:
# Check naming convention
elems = model.by_type("IfcBorehole")
passed, failed = [], []
for elem in elems:
    if re.match( r'^bh_\d{3}$', elem.Name):
        passed.append(elem)
    else:
        failed.append(elem)

In [10]:
# Unique Names
elems =model.by_type("IfcBorehole")
counter = Counter([i.Name for i in model.by_type("IfcBorehole")])
passed, failed = [], []
for elem in elems:
    if counter[elem.Name]==1:
        passed.append(elem.Name)
    else:
        failed.append(elem.Name)